TEXT PROCESSING

In [219]:
#Import Library Text Processing
import re #librar Regular Expression
import pandas as pd #library data manipulation
import sqlite3 #library database

In [220]:
#Read Data.csv
df = pd.read_csv('data.csv', encoding='latin-1')
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


- FUNGSI-FUNGSI TEXT CLEANSING DATA

In [221]:
# Mengubah semua huruf menjadi huruf kecil
def lowercase(s):
    return s.lower()

In [222]:
# Hapus Semua Tanda Baca dan Kata yang Tidak Diperlukan
def punctuation(s):
    s = re.sub('[^0-9a-zA-Z]+', ' ', s) #menghilangkan semua karakter yang bukan huruf atau angka dan menggantinya dengan spasi.
    s = re.sub('^rt',' ', s) #menghapus awalan rt
    s = re.sub(r'\d+', '', s) #menghapus semua angka
    s = re.sub('user',' ', s) #menghapus kata 'user'
    s = re.sub(r':', ' ', s) #menggantikan karakter : dengan spasi
    s = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ', s) #menghapus semua URL
    s = re.sub('url',' ', s) #menghapus kata 'url' 
    s = re.sub(' +', ' ', s) #menggantikan satu atau lebih spasi berturut-turut dengan satu spasi 
    s = re.sub('\n',' ',s) #menggantikan karakter newline (\n) dengan spasi 
    s = re.sub(r'pic.twitter.com.[\w]+', ' ', s) #menghapus semua tautan Twitter (pic.twitter.com)
    return s

-- NORMALISAI KATA ALAY DENGAN KAMUS ALAY

In [223]:
#Membuat Koneksi ke database
conn = sqlite3.connect('kamusalay_database.db')

In [224]:
# Read New_kamusalay.csv
kamusalay = pd.read_csv('new_kamusalay.csv',encoding='latin-1',names=['alay','normal'])

# Kamusalay dan database connected
kamusalay.to_sql('kamusalay', conn, if_exists='replace',index=False)
kamusalay=pd.DataFrame(kamusalay)
kamusalay

,alay,normal
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit


In [225]:
# Read atau query data kamusalay pada database
db = sqlite3.connect('kamusalay_database.db', check_same_thread = False)
q_kamusalay = 'SELECT * FROM kamusalay'
t_kamusalay = pd.read_sql_query(q_kamusalay, db)

In [226]:
# Normalisasi Kata Alay
alay_dict = dict(zip(t_kamusalay['alay'], t_kamusalay['normal']))
def normalization(s):
    for word in alay_dict:
        return ' '.join([alay_dict[word] if word in alay_dict else word for word in s.split(' ')])

-- ABUSIVE WORDS REMOVAL DENGAN DATA ABUSIVE

In [227]:
#Membuat Koneksi ke database
conn = sqlite3.connect('abusive_database.db')

In [228]:
# Read Abusive.csv
abusive = pd.read_csv('abusive.csv')

# Abusive Word dan database connected
abusive.to_sql('abusive', conn, if_exists='replace', index=False)
abusive=pd.DataFrame(abusive)
abusive

,ABUSIVE
0,alay
1,ampas
2,buta
3,keparat
4,anjing
...,...
120,rezim
121,sange
122,serbet
123,sipit


In [229]:
# Mengambil Kembali Abusive
db = sqlite3.connect('abusive_database.db', check_same_thread=False)
q_abusive = 'SELECT * FROM abusive'
t_abusive = pd.read_sql_query(q_abusive, db)

In [230]:
# Fungsi Removal Kata Abusive
def abusive_removal(s):
    abusive_words = set(t_abusive['ABUSIVE'])
    words = s.split()
    filtered_words = [word for word in words if word not in abusive_words]
    return ' '.join(filtered_words)

-- STOPWORDS REMOVAL DENGAN DATA ABUSIVE

In [231]:
#Membuat Koneksi ke database
conn = sqlite3.connect('stopword_database.db')

In [232]:
# Read Abusive.csv
stopword = pd.read_csv('stopwordbahasa.csv')

# Abusive Word dan database connected
stopword.to_sql('stopword', conn, if_exists='replace', index=False)
stopword=pd.DataFrame(stopword)
stopword

,STOPWORD
0,ada
1,adalah
2,adanya
3,adapun
4,agak
...,...
769,nih
770,wkwk
771,haha
772,eh


In [233]:
# Mengambil Kembali Stopword
db = sqlite3.connect('stopword_database.db', check_same_thread=False)
q_stopword = 'SELECT * FROM stopword'
t_stopword = pd.read_sql_query(q_stopword, db)

In [234]:
# Fungsi Stopword Removal
def stopword_removal(s):
    stopword_words = set(t_stopword['STOPWORD'])
    words = s.split()
    filtered_words = [word for word in words if word not in stopword_words]
    return ' '.join(filtered_words)

- TEXT CLEANSING

In [235]:
# Copy Dataset
data = df.copy()
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [236]:
#Membuat Fungsi Text Cleansing
def text_cleansing(s):
    s = lowercase(s)
    s = punctuation(s)
    s = normalization(s)
    s = abusive_removal(s)
    s = stopword_removal(s)
    return s

In [237]:
data['Tweet'] = data['Tweet'].apply(text_cleansing)
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,cowok berusaha melacak perhatian lantas remehk...,1,1,1,0,0,0,0,0,1,1,0,0
1,telat tau bergaul cigax jifla calis licew,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang berpikir percaya tuhan jatuh berkali ka...,0,0,0,0,0,0,0,0,0,0,0,0
3,ku tau matamu,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum dongoknya,1,1,0,1,1,0,0,0,0,0,1,0


In [238]:
#Membandingkan Tweet lama dan Tweet yang sudah di Cleansing
print('Tweet lama: {}'.format(df['Tweet'].iloc[27]))
print('\t')
print('Tweet baru: {}'.format(data['Tweet'].iloc[27]))

Tweet lama: menurutku pintu sorga ada yaitu pintu sorga yang asli dan pintu hatimu modusbanget
	
Tweet baru: menurutku pintu sorga pintu sorga asli pintu hatimu modus


- MENYIMPAN DATA BERSIH

In [239]:
#Koneksi data yang sudah dibersihkan ke Database
data.to_sql('data', conn, if_exists='replace',index=False)
data=pd.DataFrame(data)
data

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,cowok berusaha melacak perhatian lantas remehk...,1,1,1,0,0,0,0,0,1,1,0,0
1,telat tau bergaul cigax jifla calis licew,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang berpikir percaya tuhan jatuh berkali ka...,0,0,0,0,0,0,0,0,0,0,0,0
3,ku tau matamu,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum dongoknya,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,berbicara ndasmu sekate,1,1,1,0,0,0,1,0,0,1,0,0
13165,kasur enak,0,1,0,0,0,0,0,0,0,0,0,0
13166,hati hati bosan duh xf xf x xaa,0,0,0,0,0,0,0,0,0,0,0,0
13167,bom real mudah terdeteksi bom terkubur dahsyat...,0,0,0,0,0,0,0,0,0,0,0,0


In [240]:
#Menutup Database
conn.commit()
conn.close()

In [241]:
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,cowok berusaha melacak perhatian lantas remehk...,1,1,1,0,0,0,0,0,1,1,0,0
1,telat tau bergaul cigax jifla calis licew,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang berpikir percaya tuhan jatuh berkali ka...,0,0,0,0,0,0,0,0,0,0,0,0
3,ku tau matamu,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum dongoknya,1,1,0,1,1,0,0,0,0,0,1,0
